# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("CityWeather.csv")
df.head()

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,vardo,70.37,31.11,37.53,80,1013,13.18,7
1,puerto ayora,-0.74,-90.35,83.52,62,1012,3.36,20
2,busselton,-33.65,115.33,54.64,38,1007,10.40,2
3,ostrovnoy,68.05,39.51,26.96,93,1018,13.02,2
4,khatanga,71.97,102.50,14.29,97,1014,0.78,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[["latitude", "longitude"]].astype(float)
humidity = df["humidity"]

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
mask1 = (df.temperature >= 50) & (df.temperature < 70)
mask2 = df.wind_speed < 10
mask3 = (df.latitude > -40) & (df.latitude < 60)
mask4 = (df.humidity < 20)
mask5 = mask1 & mask2 & mask3 & mask4

goodSpots = df.loc[mask5].reset_index(drop=True)
goodSpots

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,hami,42.80,93.45,67.55,17,1010,7.72,83
1,hukuntsi,-24.00,21.75,66.29,18,1019,6.69,0
2,custoias,41.11,-7.32,64.00,8,1021,3.06,76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
fig = gmaps.figure(map_type="HYBRID")

coordinates = goodSpots[["latitude", "longitude"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
def getCountry(lat, long):
    rtnCountry = ""
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"key": g_key}
    lat = goodSpots["latitude"][indx]
    lng = goodSpots["longitude"][indx]
    params['latlng'] = f"{lat},{lng}"
    spot = requests.get(base_url, params=params)
    thing = spot.json()
    thing["results"][0]["address_components"]

    for addressComp in thing["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]            
    return rtnCountry

In [9]:
countries = []
for indx, row in goodSpots.iterrows():
    countries.append(getCountry(row.latitude, row.longitude))

In [10]:
goodSpots["country"] = countries

In [11]:
goodSpots.head()

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country
0,hami,42.80,93.45,67.55,17,1010,7.72,83,China
1,hukuntsi,-24.00,21.75,66.29,18,1019,6.69,0,Botswana
2,custoias,41.11,-7.32,64.00,8,1021,3.06,76,Portugal


In [12]:
def getHotel(lat, long):
    params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key}
    
    params["location"] = f"{lat},{long}"
    
    hotelUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(hotelUrl, params=params)
    name_address = name_address.json()
    
    try:
        hotelUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        name_address = requests.get(hotelUrl, params=params)
        name_address = name_address.json()
        rtnHotel = name_address["results"][0]["name"]
        return rtnHotel
    except IndexError:
        print("No Hotel")
        pass

In [13]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

hotel = []

for index, row in goodSpots.iterrows():
    lat = row["latitude"]
    lng = row["longitude"]
    hotel.append(getHotel(lat, lng))

In [14]:
goodSpots["hotel"] = hotel

In [15]:
goodSpots.head()

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country,hotel
0,hami,42.80,93.45,67.55,17,1010,7.72,83,China,哈密环球大酒店
1,hukuntsi,-24.00,21.75,66.29,18,1019,6.69,0,Botswana,Entabeni Hotel
2,custoias,41.11,-7.32,64.00,8,1021,3.06,76,Portugal,Douro Yachts & Chalets


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{cities}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in goodSpots.iterrows()]
locations = goodSpots[["latitude", "longitude"]]

In [17]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=goodSpots.humidity, dissipating=True)

# Add layer
fig.add_layer(heat_layer)

#add points
coordinates = goodSpots[["latitude", "longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…